In [ ]:
#  Feature Extraction for Flickr30k Images using EfficientNetB5

import os
import numpy as np
import pickle
from tqdm import tqdm
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

# Dizin Ayarları
IMAGE_DIR = "data/flickr30k/images"
OUTPUT_FEATURES_PATH = "models/features_flickr30k.pkl"

#  EfficientNetB5 Modeli
base_model = EfficientNetB5(weights="imagenet", include_top=False, input_shape=(456, 456, 3))
x = GlobalAveragePooling2D()(base_model.output)
model = Model(inputs=base_model.input, outputs=x)
print("EfficientNetB5 model hazır.")

#  Özellik Çıkarımı
features = {}
image_files = [f for f in os.listdir(IMAGE_DIR) if f.endswith(".jpg") or f.endswith(".jpeg") or f.endswith(".png")]

for img_name in tqdm(image_files):
    img_path = os.path.join(IMAGE_DIR, img_name)
    image = load_img(img_path, target_size=(456, 456))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    feature_vector = model.predict(image, verbose=0)

    image_id = os.path.splitext(img_name)[0]  # "1000092795.jpg" → "1000092795"
    features[image_id] = feature_vector

#  Özellikleri Kaydet
with open(OUTPUT_FEATURES_PATH, "wb") as f:
    pickle.dump(features, f)

print(f"\nToplam {len(features)} görsel için özellik çıkarımı tamamlandı.")
